In [5]:
# Utils
from tqdm.notebook import tqdm
import generate_datasets
import pandas as pd
import random
import os

# Metrics and model selection
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from imblearn.metrics import geometric_mean_score

# Models
from sklearn.neighbors import KNeighborsClassifier
from sklearn_lvq import GlvqModel
from sklearn.svm import SVC

In [6]:
SCALING_METHODS = ['original', 'MM', 'SS']
DATASETS_DIR = "scaled_datasets"
RANDOM_STATE = 10
N_FOLDS = 5

In [7]:
random.seed(RANDOM_STATE)

In [8]:
_, weights = generate_datasets.get_imbalance()
datasets = []
for i in range(len(os.listdir(DATASETS_DIR))):
  base_path = f"{DATASETS_DIR}/dataset_{i+1}"
  levels = []
  for j in range(len(weights)):
    weight = weights[j]
    path = f"{base_path}/w_{weight:.3f}"
    csvs = os.listdir(path)
    methods = []
    for k in range(len(SCALING_METHODS)):
      dataset = SCALING_METHODS[k] + ".csv"
      methods.append(pd.read_csv(f"{path}/{dataset}"))
    levels.append(methods)
  datasets.append(levels)

In [9]:
models = {
  'KNN': KNeighborsClassifier(n_neighbors=5),
  'GLQV': GlvqModel(prototypes_per_class=1, max_iter=2500, gtol=1e-5, beta=5, random_state=RANDOM_STATE)
}

In [10]:
scores = {
  'accuracy': accuracy_score,
  'f1_score': f1_score,
  'g_mean': geometric_mean_score,
  'roc_auc': roc_auc_score
}

In [11]:
def calculate_score(y_true, y_pred):
  results = [(name, func(y_true, y_pred)) for name, func in scores.items()]
  return dict(results)

In [12]:
results = {}
folds = StratifiedKFold(n_splits=N_FOLDS, random_state=RANDOM_STATE, shuffle=True)

for name, model in tqdm(models.items(), total=len(models)):
  results[name] = []
  
  for i in tqdm(range(10), leave=False): # len(datasets)
    dataset_results = []
    for j in tqdm(range(len(weights)), leave=False):
      level_results = []
      for k in tqdm(range(len(SCALING_METHODS)), leave=False):
        dataset = datasets[i][j][k]
        X = dataset.iloc[:, :-1]
        y = dataset.iloc[:, -1]
        model_scores = []
        for train_index, test_index in tqdm(folds.split(X, y), total=N_FOLDS, leave=False):
          X_train = X.iloc[train_index]
          X_test = X.iloc[test_index]
          y_train = y.iloc[train_index]
          y_test = y.iloc[test_index]
          models[name].fit(X_train, y_train)
          model_scores.append(calculate_score(y_test, models[name].predict(X_test)))
        level_results.append(model_scores)
      dataset_results.append(level_results)
    results[name].append(dataset_results)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [14]:
models_frames = []
for model_name in models.keys():
  datasets_frames = []
  for i in range(10): # len(datasets)
    weights_frames = []
    for w in range(len(weights)):
      score_frames = [pd.DataFrame(scores, index=[f"fold {i+1}" for i in range(5)]) for scores in results[model_name][i][w]]
      weight_results = pd.concat(score_frames, axis=1, keys=SCALING_METHODS)
      weights_frames.append(weight_results)
    dataset_results = pd.concat(weights_frames, axis=1, keys=[f"{weight:.3f}" for weight in weights])
    datasets_frames.append(dataset_results)
  model_results = pd.concat(datasets_frames, axis=1, keys=[f"dataset {d+1}" for d in range(10)])
  models_frames.append(model_results)
final_result = pd.concat(models_frames, axis=1, keys=models.keys())
final_result.to_csv("results.csv")

final_result

KNN                                                             \
       dataset 1                                                              
           0.500                                                              
        original                                     MM                       
        accuracy  f1_score    g_mean   roc_auc accuracy  f1_score    g_mean   
fold 1     0.950  0.950000  0.950048  0.950095    0.990  0.990000  0.990050   
fold 2     0.950  0.950980  0.949837  0.949895    0.985  0.985075  0.985037   
fold 3     0.965  0.965854  0.964724  0.964846    0.995  0.995025  0.995037   
fold 4     0.975  0.975124  0.975036  0.975048    0.995  0.995025  0.995037   
fold 5     0.965  0.966184  0.964413  0.964746    0.995  0.995074  0.994937   

                                     ...       GLQV                     \
                                     ... dataset 10                      
                                     ...      0.007                      
                       SS            ...   original                 MM   
         roc_auc accuracy  f1_score  ...     g_mean   roc_auc accuracy   
fold 1  0.990099    0.990  0.990000  ...        0.0  0.497475     0.99   
fold 2  0.985049    0.985  0.985075  ...        0.0  0.500000     0.99   
fold 3  0.995050    0.995  0.995025  ...        0.0  0.500000     0.99   
fold 4  0.995050    0.995  0.995025  ...        0.0  0.500000     0.99   
fold 5  0.994949    0.995  0.995074  ...        0.0  0.500000     0.99   

                                                                   
                                                                   
                                                                   
                                      SS                           
        f1_score g_mean roc_auc accuracy  f1_score g_mean roc_auc  
fold 1  0.994975    0.0     0.5     0.99  0.994975    0.0     0.5  
fold 2  0.994975    0.0     0.5     0.99  0.994975    0.0     0.5  
fold 3  0.994975    0.0     0.5     0.99  0.994975    0.0     0.5  
fold 4  0.994975    0.0     0.5     0.99  0.994975    0.0     0.5  
fold 5  0.994975    0.0     0.5     0.99  0.994975    0.0     0.5  

[5 rows x 2400 columns]